In [4]:
import re, os, glob, nltk
import pandas as pd
import feather
from collections import Counter
from nltk import bigrams, ngrams, trigrams
from nltk.stem import WordNetLemmatizer
from nltk.collocations import *
from nltk.tokenize import word_tokenize

In [5]:
stop_words = ['myself', 'i', 'im','my', 'you','me','we','youre','youve','weve','gon','na', 'nine','hundred','thousand','two','seven','eight','me', 'our', 'ours', 'ourselves', 'your', 'yours',
    'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she',
    "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself',
    'them', 'their', 'theirs', 'themselves', 'what', 'which',
    'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am',
    'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has',
    'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the',
    'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of',
    'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
    'through', 'during', 'before', 'after', 'above', 'below', 'to',
    'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under',
    'again', 'further', 'then', 'once', 'here', 'there', 'when', 
    'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
    'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own',
    'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just',
    'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're',
    've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't",
    'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't",
    'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't",
    'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won',
    "won't", 'wouldn', "wouldn't", "fool", "ive", "dont", "youd", "would", "theyre", "they", "else", "youll"]

In [9]:
def get_ngrams(n, path, filter_word):
    lemmatizer = WordNetLemmatizer()
    lt = [x[0] for x in os.walk(path)]

    dict_pronoun_words = {}
    bigrams_dict = {}

    # Ngrams with 'creature' as a member
    think_filter = lambda *w: 'think' not in w

    for ftr_dir in lt[1:]:
        # Gets pronoun from file name
        pronoun = ftr_dir[21:]

        # Uses the json_dir variable to navigate to the dataset directory and 'get' all json files
        ftr_pattern = os.path.join(ftr_dir, '*.ftr')

        file_list = glob.glob(ftr_pattern)
        
        for i in file_list:
            bigram_measures = nltk.collocations.BigramAssocMeasures()
            flat_list = []
            read_ftr = pd.read_feather(i, columns=None, use_threads=True)
            ls = read_ftr["text"].tolist()

            flat_list = flat_list + [item for sublist in ls for item in sublist]
            dict_pronoun_words[pronoun] = flat_list
        
        if n < 3:
            # Bigrams
            
            for i in dict_pronoun_words:
                tokens = dict_pronoun_words.get(i)
                tokens = ' '.join([lemmatizer.lemmatize(w) for w in tokens])
                tokens= tokens.split()
                tokens = [wrd for wrd in tokens if wrd.lower() not in stop_words]
                finder = BigramCollocationFinder.from_words(tokens)
                # only bigrams that appear 3+ times
                finder.apply_freq_filter(3)
                # Ngrams with 'creature' as a member
                ngram_word_filter = lambda *w: filter_word not in w

                # only bigrams that contain the filter word
                finder.apply_ngram_filter(ngram_word_filter)
                #text_bigrams = bigrams(tokens)
                bigrams_dict[i] = finder.nbest(bigram_measures.likelihood_ratio, 10)
        if n == 3:
            for i in dict_pronoun_words:
                tokens = dict_pronoun_words.get(i)
                tokens = ' '.join([lemmatizer.lemmatize(w) for w in tokens])
                tokens= tokens.split()
                tokens = [wrd for wrd in tokens if wrd.lower() not in stop_words]
                finder = TrigramCollocationFinder.from_words(tokens)
                # only bigrams that appear 3+ times
                finder.apply_freq_filter(3)
                # Ngrams with 'creature' as a member
                ngram_word_filter = lambda *w: filter_word not in w

                # only bigrams that contain the filter word
                finder.apply_ngram_filter(ngram_word_filter)
                #text_bigrams = bigrams(tokens)
                bigrams_dict[i] = finder.nbest(trigram_measures.likelihood_ratio, 10)

        else:
            for i in dict_pronoun_words:
                tokens = dict_pronoun_words.get(i)

            list_of_ngrams = []
            sixgrams = list_of_ngrams.append(ngrams(tokens, n))

            return list_of_ngrams
    if n <= 3:
        return bigrams_dict

In [8]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [25]:
ftr_dir = '../all-en-ftrs/'

def get_ngrams_new(n, path, filter_word, flat_list):
    lemmatizer = WordNetLemmatizer()

    dict_pronoun_words = {}
    bigrams_dict = {}
    
    if n < 3:
        # Bigrams
        for i in dict_pronoun_words:
            tokens = dict_pronoun_words.get(i)
            tokens = ' '.join([lemmatizer.lemmatize(w) for w in tokens])
            tokens= tokens.split()
            tokens = [wrd for wrd in tokens if wrd.lower() not in stop_words]
            finder = BigramCollocationFinder.from_words(tokens)
            # only bigrams that appear 3+ times
            finder.apply_freq_filter(3)
            # Ngrams with 'creature' as a member
            ngram_word_filter = lambda *w: filter_word not in w

            # only bigrams that contain the filter word
            finder.apply_ngram_filter(ngram_word_filter)
            #text_bigrams = bigrams(tokens)
            bigrams_dict[i] = finder.nbest(bigram_measures.likelihood_ratio, 10)
            
    if n == 3:
        flat_list = [wrd for wrd in flat_list if wrd.lower() not in stop_words]
        finder = TrigramCollocationFinder.from_words(flat_list)

        # only bigrams that appear 3+ times
        finder.apply_freq_filter(3)
        # Ngrams with 'creature' as a member
        ngram_word_filter = lambda *w: filter_word not in w

        # only bigrams that contain the filter word
        finder.apply_ngram_filter(ngram_word_filter)
        #text_bigrams = bigrams(tokens)
        trigrams = finder.nbest(trigram_measures.likelihood_ratio, 20)

    if n <= 3:
        return trigrams

In [ ]:
# Uses the json_dir variable to navigate to the dataset directory and 'get' all json files
ftr_pattern = os.path.join(ftr_dir, '*.ftr')
file_list = glob.glob(ftr_pattern)
flat_list = []
for i in file_list:  
    read_ftr = pd.read_feather(i, columns=None, use_threads=True)
    ls = read_ftr["text"].tolist()
    
    flat_list += [item for sublist in ls for item in sublist]

In [26]:
trigrams_dic_beieve = get_ngrams_new(3, '../ftrs/Pronouns/', 'believe', flat_list)

In [27]:
print(trigrams_dic_beieve)

[('believe', 'fact', 'checking'), ('fact', 'checking', 'believe'), ('via', 'youtube', 'believe'), ('believe', 'social', 'medium'), ('social', 'medium', 'believe'), ('stop', 'spreading', 'believe'), ('believe', 'new', 'york'), ('press', 'conference', 'believe'), ('believe', 'cnn', 'msnbc'), ('cnn', 'msnbc', 'believe'), ('believe', 'corona', 'virus'), ('corona', 'virus', 'believe'), ('believe', 'president', 'trump'), ('president', 'trump', 'believe'), ('right', 'wing', 'believe'), ('believe', 'right', 'wing'), ('believe', 'donald', 'trump'), ('united', 'state', 'believe'), ('believe', 'united', 'state'), ('believe', 'dr', 'fauci')]


In [29]:
trigrams_dic_think = get_ngrams_new(3, '../ftrs/Pronouns/', 'think', flat_list)

In [30]:
print(trigrams_dic_think)

[('think', 'fact', 'checking'), ('fact', 'checking', 'think'), ('via', 'youtube', 'think'), ('think', 'social', 'medium'), ('social', 'medium', 'think'), ('stop', 'spreading', 'think'), ('think', 'stop', 'spreading'), ('think', 'new', 'york'), ('think', 'press', 'conference'), ('press', 'conference', 'think'), ('think', 'corona', 'virus'), ('think', 'cnn', 'msnbc'), ('cnn', 'msnbc', 'think'), ('corona', 'virus', 'think'), ('white', 'house', 'think'), ('think', 'white', 'house'), ('think', 'president', 'trump'), ('president', 'trump', 'think'), ('think', 'right', 'wing'), ('right', 'wing', 'think')]


In [32]:
trigrams_dic_know = get_ngrams_new(3, '../ftrs/Pronouns/', 'know', flat_list)

In [33]:
print(trigrams_dic_know)

[('know', 'fact', 'checking'), ('fact', 'checking', 'know'), ('know', 'via', 'youtube'), ('via', 'youtube', 'know'), ('know', 'social', 'medium'), ('social', 'medium', 'know'), ('know', 'stop', 'spreading'), ('stop', 'spreading', 'know'), ('new', 'york', 'know'), ('know', 'new', 'york'), ('press', 'conference', 'know'), ('know', 'cnn', 'msnbc'), ('cnn', 'msnbc', 'know'), ('know', 'corona', 'virus'), ('corona', 'virus', 'know'), ('white', 'house', 'know'), ('know', 'white', 'house'), ('know', 'president', 'trump'), ('president', 'trump', 'know'), ('know', 'right', 'wing')]


# old

In [22]:
bigrams_dic = get_ngrams(2, '../all-en-ftrs/Words', "think")

In [5]:
trigrams_dic_think = get_ngrams(3, '../all-en-ftrs/Words', "think")

NameError: name 'trigram_measures' is not defined

In [32]:
for i in trigrams_dic_think:
    print(i, "\n", trigrams_dic_think.get(i))

I 
 [('think', 'president', 'trump'), ('think', 'trump', 'supporter'), ('cant', 'help', 'think'), ('people', 'still', 'think'), ('think', 'thats', 'great'), ('think', 'generally', 'dumb'), ('think', '𝐜𝐫𝐢𝐭𝐢𝐜𝐚𝐥𝐥𝐲', 'gullible'), ('acted', 'differently', 'think'), ('parent', 'think', '𝐜𝐫𝐢𝐭𝐢𝐜𝐚𝐥𝐥𝐲'), ('real', 'think', 'generally')]
Me 
 [('know', 'people', 'think')]
My 
 [('father', 'think', 'nothing'), ('think', 'nothing', 'flu'), ('elderly', 'father', 'think')]
They 
 [('parent', 'think', '𝐜𝐫𝐢𝐭𝐢𝐜𝐚𝐥𝐥𝐲'), ('although', 'think', 'great'), ('think', '𝐜𝐫𝐢𝐭𝐢𝐜𝐚𝐥𝐥𝐲', 'gullible'), ('think', 'theyve', 'made'), ('people', 'still', 'think'), ('dja', 'ever', 'think'), ('prank', 'parent', 'think'), ('think', 'great', 'nation'), ('one', 'people', 'think'), ('ever', 'think', 'theyve')]
Us 
 [('tell', 'u', 'think'), ('think', 'president', 'trump'), ('think', 'trump', 'supporter'), ('people', 'still', 'think')]
We 
 [('nope', 'vigilant', 'think'), ('vigilant', 'think', 'sir'), ('think', 'sir', 'realdonaldtru

In [10]:
s_dic_believe = get_ngrams(3, '../all-en-ftrs/Words', "believe")

In [12]:
for i in s_dic_believe:
    print(i, "\n", s_dic_believe.get(i))

I 
 [('believe', 'president', 'trump'), ('saver', 'europe', 'believe'), ('still', 'cant', 'believe'), ('cant', 'believe', 'people'), ('cant', 'believe', 'even'), ('europe', 'believe', 'virus'), ('believe', 'anything', 'say'), ('believe', 'virus', 'made'), ('want', 'believe', 'theory'), ('starting', 'believe', 'theory')]
Me 
 []
My 
 [('people', 'believe', 'theory')]
They 
 [('cant', 'believe', 'anything'), ('believe', 'everything', 'say'), ('want', 'u', 'believe'), ('believe', 'everything', 'tell'), ('saver', 'europe', 'believe'), ('believe', 'everything', 'see'), ('believe', 'u', 'intelligence'), ('believe', 'word', 'say'), ('care', 'people', 'believe'), ('people', 'believe', 'theory')]
Us 
 [('saver', 'europe', 'believe'), ('believe', 'u', 'intelligence'), ('europe', 'believe', 'virus'), ('want', 'u', 'believe'), ('believe', 'virus', 'made'), ('believe', 'virus', 'create'), ('people', 'believe', 'virus'), ('believe', 'black', 'plaguert'), ('believe', 'everything', 'tell'), ('want', '

In [13]:
trigrams_dic_know = get_ngrams(3, '../all-en-ftrs/Words', "know")

In [15]:
for i in trigrams_dic_know:
    print(i, "\n", trigrams_dic_know.get(i))

I 
 [('know', 'sound', 'like'), ('know', 'april', 'day'), ('april', 'day', 'know'), ('know', 'whats', 'going'), ('really', 'want', 'know'), ('miraculously', 'know', 'whats'), ('know', 'whats', 'real'), ('know', 'whats', 'true'), ('id', 'like', 'know'), ('know', 'holded', 'wuhan')]
Me 
 [('please', 'let', 'know'), ('thanks', 'letting', 'know'), ('know', 'people', 'think')]
My 
 []
They 
 [('know', 'many', 'people'), ('want', 'know', 'truth'), ('wouldnt', 'even', 'know'), ('someone', 'know', 'motive'), ('know', 'true', 'hid'), ('know', 'motive', 'ever'), ('wide', 'someone', 'know'), ('docked', 'know', 'brazen'), ('story', 'know', 'true'), ('people', 'know', 'truth')]
Us 
 [('let', 'u', 'know'), ('want', 'u', 'know'), ('know', 'whats', 'going'), ('none', 'u', 'know'), ('many', 'u', 'know'), ('letting', 'u', 'know'), ('know', 'everything', 'american'), ('trump', 'know', 'everything'), ('virus', 'trump', 'know')]
We 
 [('know', 'whats', 'coming'), ('police', 'personnel', 'know'), ('need', '

In [7]:
import os
from nltk.corpus.reader.plaintext import PlaintextCorpusReader

I_corpusdir = '../textFiles/Pronoun/Large/Stripped/I'
I_corpus = PlaintextCorpusReader(I_corpusdir, '.*')

you_corpusdir = '../textFiles/Pronoun/Large/Stripped/you'
you_corpus = PlaintextCorpusReader(you_corpusdir, '.*')

me_corpusdir = '../textFiles/Pronoun/Large/Stripped/me'
me_corpus = PlaintextCorpusReader(me_corpusdir, '.*')

my_corpusdir = '../textFiles/Pronoun/Large/Stripped/my'
my_corpus = PlaintextCorpusReader(my_corpusdir, '.*')

they_corpusdir = '../textFiles/Pronoun/Large/Stripped/they'
they_corpus = PlaintextCorpusReader(they_corpusdir, '.*')

us_corpusdir = '../textFiles/Pronoun/Large/Stripped/us'
us_corpus = PlaintextCorpusReader(us_corpusdir, '.*')

we_corpusdir = '../textFiles/Pronoun/Large/Stripped/we'
we_corpus = PlaintextCorpusReader(we_corpusdir, '.*')


for infile in sorted(you_corpus.fileids()):
    print(infile)

stripped_You.txt


In [7]:
import nltk
from nltk.collocations import *

bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

# Ngrams with 'creature' as a member
creature_filter = lambda *w: 'i' not in w

## Bigrams
I_finder = BigramCollocationFinder.from_words(I_corpus.words(I_corpus.fileids()))
# only bigrams that appear 3+ times
I_finder.apply_freq_filter(3)
# only bigrams that contain 'creature'
I_finder.apply_ngram_filter(creature_filter)
# return the 10 n-grams with the highest PMI
print(I_finder.nbest(bigram_measures.likelihood_ratio, 10))

NameError: name 'I_corpus' is not defined

In [13]:
def find_bigrams(corpus):
    stop_words = ['myself', 'our', 'ours', 'ourselves', 'your', 'yours',
    'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she',
    "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself',
    'them', 'their', 'theirs', 'themselves', 'what', 'which',
    'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am',
    'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has',
    'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the',
    'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of',
    'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
    'through', 'during', 'before', 'after', 'above', 'below', 'to',
    'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under',
    'again', 'further', 'then', 'once', 'here', 'there', 'when', 
    'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
    'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own',
    'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just',
    'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're',
    've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't",
    'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't",
    'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't",
    'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won',
    "won't", 'wouldn', "wouldn't"]
    
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    
    # Bigrams
    finder = BigramCollocationFinder.from_words(corpus.words(corpus.fileids()))

    # only bigrams that appear 3+ times
    finder.apply_freq_filter(3)
    finder = [tup for tup in finder if not False in [False for wrd in tup if wrd.lower() in stop_words]]

    # only bigrams that contain 'creature'
    # finder.apply_ngram_filter(creature_filter)

    # return the 10 n-grams with the highest PMI
    return finder.nbest(bigram_measures.likelihood_ratio, 10)

In [3]:
import nltk
from nltk.collocations import *

bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

# Ngrams with 'creature' as a member
think_filter = lambda *w: 'think' not in w

## Bigrams
I_finder = BigramCollocationFinder.from_words(I_corpus)
# only bigrams that appear 3+ times
I_finder.apply_freq_filter(3)
# only bigrams that contain 'creature'
I_finder.apply_ngram_filter(creature_filter)
# return the 10 n-grams with the highest PMI
print(I_finder.nbest(bigram_measures.likelihood_ratio, 10))

TypeError: 'PlaintextCorpusReader' object is not iterable

In [14]:
#I_finder.nbest(bigram_measures.likelihood_ratio, 10)
vocab = ["I", "me", "you", "my", "they", "us", "we"]
ls = [I_corpus,
me_corpus,
you_corpus,
my_corpus,
they_corpus,
us_corpus,
we_corpus]

think_filter = lambda *w: 'think' not in w

for i,j in enumerate(ls):
    print(vocab[i], "\n", find_bigrams(j))

I 
 [('good', 'person'), ('things', 'fit'), ('im', 'good'), ('im', 'not'), ('some', 'things'), ('on', 'hulu'), ('effect', 'on'), ('person', 'some'), ('fit', 'but'), ('hulu', 'im')]
me 
 [('this', 'is'), ('want', 'to'), ('i', 'am'), ('lamestream', 'media'), ('yours', 'astound'), ('sounds', 'like'), ('if', 'you'), ('rofessional', 'tweets'), ('please', 'tell'), ('trying', 'to')]
you 
 [('this', 'is'), ('need', 'to'), ('want', 'to'), ('shame', 'on'), ('fact', 'checking'), ('going', 'to'), ('trying', 'to'), ('she', 'does'), ('gon', 'na'), ('talking', 'about')]
my 
 [('angel', 'read'), ('expose', 'them'), ('an', 'angel'), ('read', 'twitter'), ('they', 'say'), ('them', 'they'), ('say', 'im'), ('im', 'an'), ('twitter', 'and'), ('and', 'facebook')]
they 
 [('expose', 'them'), ('say', 'im'), ('not', 'expose'), ('them', 'say'), ('used', 'to'), ('angel', 'read'), ('and', 'used'), ('third', 'one'), ('an', 'angel'), ('my', 'twitter')]
us 
 [('in', 'the'), ('scientific', 'dat'), ('game', 'escalates')